In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
%matplotlib inline

In [3]:

if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

# Import and normalize data

In [4]:
train_origin = loadmat('Z:/ricardo/data/Project/State_classification/Reviewed_labels/brain_state_trainset.mat')
val_origin = loadmat('Z:/ricardo/data/Project/State_classification/Reviewed_labels/brain_state_valset.mat')

In [5]:
from sklearn.preprocessing import RobustScaler

Normalize movement

In [6]:
x_norm = train_origin['under_train_movement'].flatten().reshape(-1,1)
moveScaler = RobustScaler(quantile_range=(10,90)).fit(x_norm)

In [7]:
def normalize_test(data_origin,spectr_field, mov_field, mov_scalefunc):

    #swap dimensions of spectrograms to have samples in the first dimension
    #l1 = np.shape(data_origin[spectr_field])
    #data_origin[spectr_field] = data_origin[spectr_field].reshape((l1[1],l1[0],l1[2]))

    #normalize spectral data using a quantile-based method that is not sensitive to outliers
    def normfunc(x):
        #center and normalize data based on percentiles 10 and 90
        transf = RobustScaler(quantile_range=(10,90)).fit(x)
        return transf.transform(x)


    spectral_norm = np.zeros((np.shape(data_origin[spectr_field])[1],np.shape(data_origin[spectr_field])[0],
    np.shape(data_origin[spectr_field])[2]))
    
    #normalize movement
    mov_norm = mov_scalefunc.transform(data_origin[mov_field].flatten().reshape(-1,1))
    mov_norm = np.reshape(mov_norm,(np.shape(data_origin[mov_field])))
    mov_norm = np.transpose(mov_norm) #transpose to have it in the form instances x movement profiles

    #Convert power spectral values to logarithms to minimize sekewness
    for i in range(np.shape(spectral_norm)[0]):
        spec = np.log(np.squeeze(data_origin[spectr_field][:,i,:]))
        #print(np.shape(spec))
        spec = normfunc(spec.flatten().reshape(-1,1)) #flatten the multidimensional data for normalization
        #spec = spec.flatten().reshape(-1,1)
        #spec = spec.ravel()
        #then recover the original format
        spectral_norm[i,:,:] = np.reshape(spec,(np.shape(spectral_norm)[1],np.shape(spectral_norm)[2]))
        if np.quantile(mov_norm[i,:],0.15)<0:
            mov_norm[i,:] = mov_norm[i,:]-np.quantile(mov_norm[i,:],0.15)

    #normalize movement data using the same approach, but now using whole dataset movement stats
    #mov_norm = normfunc(data_origin[mov_field].flatten().reshape(-1,1))
    #mov_norm = mov_scalefunc.transform(data_origin[mov_field].ravel())
    #mov_norm = np.reshape(mov_norm,(np.shape(data_origin[mov_field])))
    #mov_norm = np.transpose(mov_norm) #transpose to have it in the form instances x movement profiles

    return spectral_norm, mov_norm

In [8]:
spectral_norm_train, mov_norm_train = normalize_test(train_origin,'under_train_spectrograms','under_train_movement',moveScaler)
spectral_norm_val, mov_norm_val = normalize_test(val_origin, 'under_val_spectrograms', 'under_val_movement',moveScaler)

In [9]:
def oversample(data, spectra, mov, target_field,sampling='under'):
        
        from imblearn.over_sampling import RandomOverSampler
        from imblearn.under_sampling import RandomUnderSampler

        #The oversampling function only accepts instance x features format as inputs, 
        # so we need to reshape spectral data and concatenate it with movement
        X_data = np.concatenate((mov, spectra.reshape(len(mov),
                np.shape(spectra)[1]*np.shape(spectra)[2])),axis=1)
        y_data = data[target_field].ravel()

        #instantiate oversampler and apply it
        if sampling=='under':
                ros = RandomUnderSampler(random_state=34)
        elif sampling=='over':
                ros = RandomOverSampler(random_state=34)

        X_res, target_res = ros.fit_resample(X_data, y_data)

        #recover original data formats
        mov_norm_res = X_res[:,:np.shape(mov)[1]]
        spectral_norm_res = np.reshape(X_res[:, np.shape(mov)[1]:],
                (len(mov_norm_res),np.shape(spectra)[1],np.shape(spectra)[2]))

        return spectral_norm_res, mov_norm_res, target_res


In [19]:
spectral_norm_train_res, mov_norm_train_res, target_train_res = oversample(train_origin,spectral_norm_train, mov_norm_train,'under_train_target','under')
spectral_norm_val_res, mov_norm_val_res, target_val_res = oversample(val_origin,spectral_norm_val, mov_norm_val,'under_val_target','under')

## Create dataset object and random samplers

The dataset class from torch allows convenient storage and sampling of the data, particularly useful for neural network training.
First we need to create a class that will allow sampling batches of the dataset.

In [11]:
class DataStates(Dataset):
    def __init__(self, movement, spec, labels):
        self.labels = labels
        self.movement = movement
        self.spec = spec

    def __len__(self):
        return len(self.movement)

    def __getitem__(self, index):
        X1 = self.movement[index,:]
        X2 = self.spec[index,:,:]
        y = self.labels[index]
        return X1, X2, y


In [12]:
class DataStatesCUDA(Dataset):
    def __init__(self, movement, spec, labels):
        self.labels = labels
        self.movement = movement
        self.spec = spec

    def __len__(self):
        return len(self.movement)

    def __getitem__(self, index):
        X1 = self.movement[index, :,:]
        X2 = self.spec[index, :, :,:]
        y = self.labels[index]-1
        return X1, X2, y


In [20]:
#Create datasets
#move training data to GPU
l1 = np.shape(mov_norm_train_res)
l2 = np.shape(spectral_norm_train_res)

spectral_norm_train_res = torch.tensor(spectral_norm_train_res.reshape((l2[0],1,l2[1],l2[2])))

#mov_norm_trainCUDA = torch.tensor(mov_norm_train_res.reshape((l1[0],1,l1[1])),dtype=torch.float,device=dev,pin_memory=True)
spectral_norm_trainCUDA = torch.tensor(spectral_norm_train_res,dtype=torch.float)
spectral_norm_trainCUDA.pin_memory()
target_trainCUDA = torch.tensor(target_train_res,dtype=torch.long, device=dev, pin_memory=True)
data_train = DataStates(mov_norm_train_res, spectral_norm_train_res, target_train_res)
data_trainCUDA = DataStatesCUDA(mov_norm_trainCUDA, spectral_norm_trainCUDA, target_trainCUDA)
data_val = DataStates(mov_norm_val_res, spectral_norm_val_res, target_val_res)
#data_test = DataStates(mov_norm_test, spectral_norm_test, test_origin['test_target'].ravel())



C:\Users\Ricardo\AppData\Local\Temp\ipykernel_14076\3723264624.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spectral_norm_trainCUDA = torch.tensor(spectral_norm_train_res,dtype=torch.float)


RuntimeError: Can't pin tensor constructed from numpy

In [17]:
spectral_norm_trainCUDA.is_cuda

False

# Build a neural network

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

**Implement the neural network class**

In [14]:
class Net(nn.Module):
    def __init__(self, device, name=None, filter_size1d=5, filter_size=5, fc1_size=120, fc2_size=84,n_conv1=6,n_convtime=3):
        """Define the convolutional neural network.
        The parameters describing filter sizes and the number of neurons in the fully connected
        layers can be provided externally.
        """
        
        super(Net,self).__init__()
        if name:
            self.name=name
        
        #define input size of first fully connected layer depending on image resolution
        global pool1h, pool1w, pooltimew, num_conv1

        wrange = torch.tensor([2,5,10,20,50],dtype=int,device=dev)

        pool1h = torch.zeros(5, dtype=int, device=dev)
        #pool2h = torch.zeros(5, dtype=int, device=dev)
        pool1w = torch.zeros(5, dtype=int, device=dev)
        #pool2w = torch.zeros(5, dtype=int, device=dev)
        pooltimew = torch.zeros(5, dtype=int, device=dev)

        for i, w in enumerate(wrange):
            picres=torch.tensor([w*2+1,81], dtype=int, device=dev)
            timeres=torch.tensor([w*2*5+1], dtype=int, device=dev)

            #print(picres[0])
            #sizes
            pool1h[i] = picres[0]-(3-1)
            #pool2h[i] = torch.floor(((pool1h[i]-(3-1))-2)/2)+1
            pool1w[i] = picres[1]-(filter_size-1)
            pooltimew[i] = torch.floor(((timeres-(filter_size1d-1))-2)/2)+1
            #pool2w[i] = torch.floor(((pool1w[i]-(filter_size-1))-2)/2)+1
        pool1h[4] = picres[0]-(3-1)-1
        pooltimew[4] = pooltimew[4] - 1

        num_conv1 = torch.tensor(n_conv1,dtype=int, device=dev)
        #picres = torch.tensor([[101,81]]) #tensor with image resolution for spectral data

        #pool1h = torch.floor(((picres[0][0]-(filter_size-1))-2)/2)+1 #height of 1st maxpool layer output
        #pool2h = torch.floor(((pool1h-(filter_size-1))-2)/2)+1 #height of 2nd maxpool layer output
        #pool1w = torch.floor(((picres[0][1]-(filter_size-1))-2)/2)+1 #width of 1st maxpool layer output
        #pool2w = torch.floor(((pool1w-(filter_size-1))-2)/2)+1 #width of 2nd maxpool layer output
        
        #branch of spectral images
        self.conv1=nn.Conv2d(1,n_conv1,(3,filter_size)) #set a fixed number of filters of 6
        #self.pool=nn.MaxPool2d(2,2) #max-pooling parameters are fixed
        #self.conv2=nn.Conv2d(n_conv1,n_conv2,(3,filter_size)) #fixed number of filters 16

        #branch of movement
        self.convtime = nn.Conv1d(1,n_convtime,filter_size1d, device=dev) #use 3 1d convolution filters 
        self.pooltime = nn.MaxPool1d(2,2) #max-pooling
        #pooltimew = torch.floor(((torch.tensor(501)-(filter_size1d-1))-2)/2)+1 #width of the max-pool output

        #fully connected layers
        #the first layer gets the inputs of the 2D branch concatenated with those from the 1D branch
        
        self.fc1 = nn.ModuleList([])
        self.fc2 = nn.ModuleList([])
        self.fc3 = nn.ModuleList([])
        for i, w in enumerate(wrange):
            self.fc1.append(nn.Linear(int(n_conv1*pool1h[i].item()*pool1w[i].item())+int(n_convtime*pooltimew[i].item()),int(fc1_size*50/w)))
            self.fc2.append(nn.Linear(int(fc1_size*50/w),int(fc2_size*50/w))) #second fully connected layer
            self.fc3.append(nn.Linear(int(fc2_size*50/w),4)) #intermediate output layer
        
        #self.fc1=nn.Linear(int(n_conv2*pool2h.item()*pool2w.item())+int(n_convtime*pooltimew.item()),fc1_size)

        
        self.fcOut = nn.Linear(20,4) #final output layer
        

    def forward(self, x, x2):
        #x denotes the spectral inputs and x2 denotes movement inputs
        #the output of all convolution layers passes trhough a ReLu activation function
        wrange = torch.tensor([2,5,10,20,50],dtype=int,device=dev)
        xio = torch.zeros((len(x),4,5),dtype=torch.float,device=dev)

        
        for i, w in enumerate(wrange):
            
            #xi = x[:,:,51-w:51+w+1,:]
            #xi2 = x2[:,:,251-w*5:251+w*5+1]

            
            xi = F.relu(self.conv1(x[:,:,51-w:51+w+1,:]))
            #print(num_conv1, pool1h[i].item(),pool1w[i].item())
            #print(np.shape(x))
            #xi = self.pool(F.relu(self.conv2(xi)))
            
            #xi = xi.view(-1, int(num_conv1*pool1h[i].item()*pool1w[i].item()))
            #print(xi.shape)
            xi = xi.view(-1, xi.size(dim=1)*xi.size(dim=2)*xi.size(dim=3))
            st = time.time()
            xi2 = self.pooltime(F.relu(self.convtime(x2[:,:,251-w*5:251+w*5+1])))
            en = time.time()
            print(xi2.shape)
            #xi2 = xi2.reshape(np.shape(xi2)[0],np.shape(xi2)[1]*np.shape(xi2)[2])
            xi2 = xi2.reshape(xi2.size(dim=0),xi2.size(dim=1)*xi2.size(dim=2))
            
            #print(xi.dtype, xio.dtype)
        #the output of the 2 fully connect hidden-layers goes through ReLu activation
            xi = F.relu(self.fc1[i](torch.concat((xi, xi2),dim=1)))
            xi = F.relu(self.fc2[i](xi))
            xio[:,:,i] = self.fc3[i](xi) #intermediate output layer
            
            #en = time.time()
            if i==4:
                print((en-st)*1000)
        
        xf = F.softmax(self.fcOut(torch.concat((xio[:,:,0],xio[:,:,1],xio[:,:,2],xio[:,:,3],xio[:,:,4]),dim=1)),dim=1)
        
        
        return xf

## Training the neural network

### Training function

In [15]:
def trainNet(net, data_train, batch_size=10, epochs=4, epoch_start=0, printr=False, save_loss=False):
    """Function to train the neural network
        Inputs:
            net: network to train
            batch_size: size of the mini-batch used in the SGD optimization
            epochs: number of traning epochs
            epoch start: number of epochs run so far + 1. Used to keep track of total training
                epochs when the function takes an already pretrained network as input
            printr: boolean to determine whether running loss is displayed during execution
            save_loss: boolean used to indicate wheter to store information about the running loss"""
 
    net = net.float()
    net.to(dev)

    # create training loader
    trainloader = DataLoader(data_train,batch_size=int(batch_size))

    #train
    import time 
    start = time.time()
    global loss
    print('here')
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        epoch_start += 1
        running_loss = 0.0
        rloss = []
        
        for i, sample in enumerate(trainloader):
            #l1 = np.shape(sample[0])
            #l2 = np.shape(sample[1])

            #print(type(sample[0]),type(sample[1]))
            #get the inputs with the approppriate shape for the network function
            #input_mov = torch.tensor(sample[0].reshape((l1[0],1,l1[1])),dtype=torch.float,device=dev)
            #input_spec = torch.tensor(sample[1].reshape(l2[0],1,l2[1],l2[2]),dtype=torch.float,device=dev)
            #labels = torch.tensor(sample[2]-1, dtype=torch.long,device=dev)
            #print(type(input_mov))
            
            # zero the parameter gradients
            optimizer.zero_grad()

            #compute network output
            #outputs=net(input_spec,input_mov)
            #start2 = time.time()
            outputs=net(sample[1],sample[0])
            #end2 = time.time()
            #print(end2-start2)
            
            #print(np.shape(sample[2]))
            #calculate loss (criterion is a global variable initialized externaly)
            loss = criterion(outputs, sample[2])
            
            #update weights based on backpropagation
            loss.backward()
            optimizer.step()

            if save_loss:
                rloss.append(loss.item())

            if printr:
                # print statistics
                running_loss += loss.item()
                if i % 10 == 9:    # print every 10 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 10))
                    running_loss = 0.0
    
    end = time.time()

    if printr:
        print('Finished Training')
        print('training time ', end-start)
    
    if save_loss:
        return rloss

### Checkpoint function

Function used to instantiate the intialize either a new neural network or taking a pre-saved model 

In [16]:
def Checkpoint(path, name=None, filter_size1d=5, filter_size=5, fc1_size=120, fc2_size=84, lr=0.0005,n_conv1=6,n_convtime=3):
    """Function used to instantiate the intialize either a new neural network or taking a pre-saved model
            Inputs:
                path: path of the file containing model information(saved with the SaveCheckpoint function
                name: network name
                filter_size1d, filter_size, fc1_size, fc2_size are network hyperparameters
                lr: learning rate"""
                
    #instantiate neural network
    global net,criterion,optimizer,epoch_start
    
    #print(filter_size1d)
    net = Net(name, dev, filter_size1d, filter_size, fc1_size, fc2_size, n_conv1, n_convtime).float().to(dev)
    
    #create optimizer
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.1,1,0.1,0.1],dtype=torch.float,device=dev))
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.95)
    
    if path == '':
        epoch_start = 0
        #loss = torch.zeros(1,requires_grad=True)
    else:
        checkpt = torch.load(path)
        filter_size1d=int(checkpt['tuned_parameters']['filter_size1d'])
        filter_size = int(checkpt['tuned_parameters']['filter_size'])
        fc1_size = int(checkpt['tuned_parameters']['fc1_size'])
        fc2_size = int(checkpt['tuned_parameters']['fc2_size'])
        print(filter_size1d)
        net = Net(name, filter_size1d, filter_size, fc1_size, fc2_size)
        
        net.load_state_dict(checkpt['model_state_dict'])
        optimizer.load_state_dict(checkpt['optimizer_state_dict'])
        #epoch_start = checkpt['epoch']
        loss = checkpt['loss']
        net.train()

### Network test function

Next, we define a function to run to test the predictive performance of the CNN on the validation and test sets.

In [17]:
def test(net, data,batch_size=10, printr=False):
    """Function to test neural network predictions on a given dataset
            Inputs:
                net: Network to test
                sampler: Subset random sampler used to sample validation or test sets
                batch_size: size of batchs to compute predictions on
                printr: Whether to show performance results"""

    testloader = DataLoader(data,batch_size=batch_size)
    global dev 
    
    dev = torch.device("cpu")

    pool1h.to(torch.device('cpu')) 
    pool1w.to(torch.device('cpu'))
    pooltimew.to(torch.device('cpu'))
    #num_conv1.to(torch.device('cpu'))
    net = net.float()
    net.to(torch.device('cpu'))
    
    correct = 0
    total = 0
    with torch.no_grad():
        for sample in testloader:
            l1 = np.shape(sample[0])
            l2 = np.shape(sample[1])
            #reshaope input data to approppriate format
            input_mov = torch.tensor(sample[0].reshape((l1[0],1,l1[1])),dtype=torch.float)
            input_spec = torch.tensor(sample[1].reshape(l2[0],1,l2[1],l2[2]),dtype=torch.float)
            labels = sample[2]-1
            
            #print(input_spec.is_cuda,input_mov.is_cuda, next(net.parameters()).is_cuda)
            #compute predictions from the class with max output
            outputs=net(input_spec,input_mov)
            predicted = np.argmax(outputs,axis=1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    #compute confusion matrix
    cm = confusion_matrix(labels,predicted)
    
    if printr:
        print(cm)
        print('Accuracy:\nTotal:', 100*correct/total, '\nWake:',cm[0,0]/np.sum(cm[0,:]),'\nQuiet wake:',
        cm[1,1]/np.sum(cm[1,:]),'\nNREM:', cm[2,2]/np.sum(cm[2,:]),'\nREM:', cm[3,3]/np.sum(cm[3,:]))
    
    dev = torch.device("cuda:0")

    return cm, 100*correct/total, predicted, outputs

### Check training

In [21]:
dev=torch.device('cuda:0')

In [19]:
dev

device(type='cpu')

In [20]:
Checkpoint('', name='Net-Capstone', lr=0.002, n_conv1=6,n_convtime=2,fc1_size=200,fc2_size=100, filter_size=10, filter_size1d=14)


trainNet(net,data_trainCUDA, epochs=100,batch_size=100,printr=True)


here


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [22]:
name='Net-Capstone'
lr=0.002
n_conv1=6
n_convtime=2
fc1_size=200
fc2_size=100
filter_size=10
filter_size1d=14
batch_size=100
epochs=1

 #instantiate neural network
global net,criterion,optimizer,epoch_start

#print(filter_size1d)
net = Net(name, dev, filter_size1d, filter_size, fc1_size, fc2_size, n_conv1, n_convtime).float().to(dev)

#create optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.1,1,0.1,0.1],dtype=torch.float,device=dev))
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.95)


# create training loader
trainloader = DataLoader(data_trainCUDA,batch_size=batch_size)

#train
#start = time.time()
sample = torch.tensor([],dtype=float,device=dev)
sampleS = spectral_norm_trainCUDA[:100,:,:,:]
sampleM = mov_norm_trainCUDA[:100,:,:]
sampleL = target_trainCUDA[:100]
w=torch.tensor(50,dtype=int,device=dev)

convtime = nn.Conv1d(1,n_convtime,filter_size1d,device=dev) #use 3 1d convolution filters 
pooltime = nn.MaxPool1d(2,2) #max-pooling

start = time.time()
pooltime(F.relu(convtime(sampleM[:,:,251-w*5:251+w*5+1])))
end = time.time()

start2 = time.time()
outputs=net(sampleS,sampleM)
end2 = time.time()

print((end-start)*1000, (end2-start2)*1000)


torch.Size([100, 2, 4])
torch.Size([100, 2, 19])
torch.Size([100, 2, 44])
torch.Size([100, 2, 94])
torch.Size([100, 2, 243])
8.013725280761719
1043.6615943908691 83.78124237060547


In [56]:
wrange = torch.tensor([2,5,10,20,50],dtype=int,device=dev)

pool1h = torch.zeros(5, dtype=int, device=dev)
#pool2h = torch.zeros(5, dtype=int, device=dev)
pool1w = torch.zeros(5, dtype=int, device=dev)
#pool2w = torch.zeros(5, dtype=int, device=dev)
pooltimew = torch.zeros(5, dtype=int, device=dev)

for i, w in enumerate(wrange):
    picres=torch.tensor([w*2+1,81], dtype=int, device=dev)
    timeres=torch.tensor([w*2*5+1], dtype=int, device=dev)

    #print(picres[0])
    #sizes
    pool1h[i] = picres[0]-(3-1)
    #pool2h[i] = torch.floor(((pool1h[i]-(3-1))-2)/2)+1
    pool1w[i] = picres[1]-(filter_size-1)
    pooltimew[i] = torch.floor(((timeres-(filter_size1d-1))-2)/2)+1
    #pool2w[i] = torch.floor(((pool1w[i]-(filter_size-1))-2)/2)+1
pool1h[4] = picres[0]-(3-1)-1
pooltimew[4] = pooltimew[4] - 1

num_conv1 = torch.tensor(n_conv1,dtype=int, device=dev)

#branch of spectral images
conv1=nn.Conv2d(1,n_conv1,(3,filter_size),device=dev) #set a fixed number of filters of 6


#branch of movement
convtime = nn.Conv1d(1,n_convtime,filter_size1d, device=dev) #use 3 1d convolution filters 
pooltime = nn.MaxPool1d(2,2) #max-pooling


fc1 = nn.ModuleList([])
fc2 = nn.ModuleList([])
fc3 = nn.ModuleList([])
for i, w in enumerate(wrange):
    fc1.append(nn.Linear(int(n_conv1*pool1h[i].item()*pool1w[i].item())+int(n_convtime*pooltimew[i].item()),int(fc1_size*50/w)))
    fc2.append(nn.Linear(int(fc1_size*50/w),int(fc2_size*50/w))) #second fully connected layer
    fc3.append(nn.Linear(int(fc2_size*50/w),4)) #intermediate output layer

fc1.to(dev)
fc2.to(dev)
fc3.to(dev)


fcOut = nn.Linear(20,4) #final output layer


def forward(x, x2, conv1,convtime,pooltime):
    #x denotes the spectral inputs and x2 denotes movement inputs
    #the output of all convolution layers passes trhough a ReLu activation function
    xio = torch.zeros((len(x),4,5),dtype=torch.float,device=dev)

    for i, w in enumerate(wrange):
        #print(w)
        st = time.time()
        xt = pooltime(F.relu(convtime(x2[:,:,251-w*5:251+w*5+1])))
        torch.cuda.current_stream().synchronize()
        en = time.time()
        print((en-st)*1000)
        xt = xt.reshape(xt.size(dim=0),xt.size(dim=1)*xt.size(dim=2))
        xt2 = F.relu(conv1(x[:,:,51-w:51+w+1,:]))
        
        #print(xi.shape)
        xt2 = xt2.view(-1, xt2.size(dim=1)*xt2.size(dim=2)*xt2.size(dim=3))
        xt2 = F.relu(fc1[i](torch.concat((xt2, xt),dim=1)))
        xt2 = F.relu(fc2[i](xt2))

    for i, w in enumerate(wrange):
        
        #print('here')
        
        xi = F.relu(conv1(x[:,:,51-w:51+w+1,:]))
        
        #print(xi.shape)
        xi = xi.view(-1, xi.size(dim=1)*xi.size(dim=2)*xi.size(dim=3))
        st = time.time()
        xi2 = pooltime(F.relu(convtime(x2[:,:,251-w*5:251+w*5+1])))
        torch.cuda.current_stream().synchronize()
        en = time.time()
        print((en-st)*1000)
        #xi2 = xi2.reshape(np.shape(xi2)[0],np.shape(xi2)[1]*np.shape(xi2)[2])
        xi2 = xi2.reshape(xi2.size(dim=0),xi2.size(dim=1)*xi2.size(dim=2))
        
        #print(xi.is_cuda, xi2.is_cuda, xio.is_cuda)
    #the output of the 2 fully connect hidden-layers goes through ReLu activation
        xi = F.relu(fc1[i](torch.concat((xi, xi2),dim=1)))
        xi = F.relu(fc2[i](xi))
        xio[:,:,i] = fc3[i](xi) #intermediate output layer
        
        #en = time.time()
        #if i==4:
        #    print((en-st)*1000)

    #xf = F.softmax(fcOut(torch.concat((xio[:,:,0],xio[:,:,1],xio[:,:,2],xio[:,:,3],xio[:,:,4]),dim=1)),dim=1)

    #return xf

forward(sampleS,sampleM,conv1,convtime,pooltime)
        

1.0035037994384766
13.018369674682617
5.979776382446289
7.813692092895508
6.980419158935547
0.6020069122314453
1.9943714141845703
2.345561981201172
1.9996166229248047
8.974075317382812


In [ ]:

for epoch in range(epochs):  # loop over the dataset multiple times
    
    epoch_start += 1
    running_loss = 0.0
    rloss = []
    
    #sample=iter(trainloader)
    #sample = sample.next()


    #l1 = np.shape(sample[0])
    #l2 = np.shape(sample[1])

    #print(type(sample[0]),type(sample[1]))
    #get the inputs with the approppriate shape for the network function
    #input_mov = torch.tensor(sample[0].reshape((l1[0],1,l1[1])),dtype=torch.float,device=dev)
    #input_spec = torch.tensor(sample[1].reshape(l2[0],1,l2[1],l2[2]),dtype=torch.float,device=dev)
    #labels = torch.tensor(sample[2]-1, dtype=torch.long,device=dev)
    #print(type(input_mov))
    
    # zero the parameter gradients
    optimizer.zero_grad()

    #compute network output
    #outputs=net(input_spec,input_mov)
    #start2 = time.time()
    #outputs=net(sample[1],sample[0])
    outputs=net(sampleS,sampleM)
    #end2 = time.time()
    #print(end2-start2)
    
    #print(np.shape(sample[2]))
    #calculate loss (criterion is a global variable initialized externaly)
    #loss = criterion(outputs, sample[2])
    loss = criterion(outputs, sampleL)

    #update weights based on backpropagation
    loss.backward()
    optimizer.step()

    #if save_loss:
    #    rloss.append(loss.item())

    # if printr:
    #     # print statistics
    #     running_loss += loss.item()
    #     if i % 10 == 9:    # print every 10 mini-batches
    #         print('[%d, %5d] loss: %.3f' %
    #             (epoch + 1, i + 1, running_loss / 10))
    #         running_loss = 0.0

#end = time.time()